In [233]:
import pandas as pd

In [240]:
df_agg_all = pd.read_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Topics_10/Topics_0/lang_topic_sentiment_agg.csv"
)
df_agg_all = df_agg_all[df_agg_all["sentiment_label"] != "error"]

In [243]:
df_agg_all

,Unnamed: 0,month,lang,topic,sentiment_label,count
0,0,2022-02,de,0,negative,774
1,1,2022-02,de,0,neutral,871
2,2,2022-02,de,0,positive,25
3,3,2022-02,de,1,negative,545
4,4,2022-02,de,1,neutral,1645
...,...,...,...,...,...,...
3566,3566,2023-06,uk,8,neutral,262
3567,3567,2023-06,uk,8,positive,36
3568,3568,2023-06,uk,9,negative,459
3569,3569,2023-06,uk,9,neutral,1104


In [289]:
topic = "0"

df_all = (
    df_agg_all.groupby(["month", "sentiment_label"])
    .sum()["count"]
    .reset_index()
    .sort_values("month")
    .pivot(index="sentiment_label", columns="month")
)
df_all.columns = df_all.columns.droplevel(0)
df_all.columns.name = None
df_all = df_all.reset_index()

df_cur = (
    df_agg_all.query(f"topic=={topic}")
    .groupby(["month", "sentiment_label"])
    .sum()["count"]
    .reset_index()
    .sort_values("month")
    .pivot(index="sentiment_label", columns="month")
)
df_cur.columns = df_cur.columns.droplevel(0)
df_cur.columns.name = None
df_cur = df_cur.reset_index()

all_months = df_agg_all["month"].unique()

abs = []
abs_rel = []
abs_all = []
for month in all_months:
    abs.append(df_cur[month].sum())
    abs_all.append(df_all[month].sum())
    abs_rel.append(df_cur[month].sum() / df_all[month].sum())

df_cur.loc[3] = ["total"] + abs
df_cur.loc[4] = ["rel"] + abs_rel
df_all.loc[3] = ["total"] + abs_all

In [288]:
df_all

,sentiment_label,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06
0,negative,231835,1550793,1044061,991744,678736,591880,587768,693418,711745,513876,460713,472572,548759,423795,201730,80360,119572
1,neutral,97664,704419,460148,818352,611015,427077,327406,384267,378156,362118,334885,300854,377819,292517,157268,71314,57697
2,positive,34519,292727,164126,252148,154742,112787,125369,146555,151982,144361,167205,117983,160346,100639,53535,23629,20094
3,total,364018,2547939,1668335,2062244,1444493,1131744,1040543,1224240,1241883,1020355,962803,891409,1086924,816951,412533,175303,197363


In [290]:
df_cur

,sentiment_label,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06
0,negative,41705.000000,224686.000000,171093.000000,119384.000000,86974.000000,90131.000000,87237.000000,90367.000000,118744.000000,87047.000000,70133.000000,74000.00000,55638.000000,59603.000000,29315.000000,21399.000000,26804.000000
1,neutral,16581.000000,88718.000000,68424.000000,57883.000000,44868.000000,35898.000000,35478.000000,45035.000000,52134.000000,38986.000000,33379.000000,41232.00000,35521.000000,31173.000000,17057.000000,10182.000000,9859.000000
2,positive,991.000000,6428.000000,5409.000000,5096.000000,3618.000000,3037.000000,2817.000000,4397.000000,4541.000000,2901.000000,2593.000000,3477.00000,2831.000000,2407.000000,1234.000000,656.000000,783.000000
3,total,59277.000000,319832.000000,244926.000000,182363.000000,135460.000000,129066.000000,125532.000000,139799.000000,175419.000000,128934.000000,106105.000000,118709.00000,93990.000000,93183.000000,47606.000000,32237.000000,37446.000000
4,rel,0.162841,0.125526,0.146809,0.088429,0.093777,0.114042,0.120641,0.114192,0.141252,0.126362,0.110204,0.13317,0.086473,0.114062,0.115399,0.183893,0.189732


In [246]:
def get_table(topic: str):

    df_pivot_cur = (
        df_agg_all.query(f"topic=={topic}")
        .groupby(["sentiment_label", "lang"])
        .sum()
        .reset_index()[["sentiment_label", "lang", "count"]]
        .pivot(index="sentiment_label", columns="lang")
    )
    df_pivot_cur.columns = df_pivot_cur.columns.droplevel(0)
    df_pivot_cur.columns.name = None
    df_pivot_cur = df_pivot_cur.reset_index()
    df_pivot_cur["sum"] = df_pivot_cur[df_agg_all["lang"].unique()].sum(axis=1)
    df_pivot_cur.loc[3] = (
        ["sum"]
        + [df_pivot_cur[lang].sum() for lang in df_agg_all["lang"].unique()]
        + [df_pivot_cur["sum"].sum()]
    )
    all_tweets = df_agg_all["count"].sum()
    topic_tweets = (
        df_agg_all.query(f"topic=={topic}").groupby("lang").sum()["count"].sum()
    )
    return pd.DataFrame(
        data=df_pivot_cur.set_index("sentiment_label").values
        / (
            df_pivot_all.set_index("sentiment_label").values
            * (topic_tweets / all_tweets)
        ),
        index=df_pivot_all["sentiment_label"],
        columns=df_pivot_all.columns[1:],
    ).reset_index()


def print_as_table(topic: str):
    df = get_table(topic)
    all_langauges = df_agg_all["lang"].unique()
    language_mapping = {
        "de": "German",
        "en": "English",
        "fr": "French",
        "es": "Spanish",
        "it": "Italian",
        "uk": "Ukrainian",
        "ru": "Russian",
    }

    print(
        "\\begin{table}[H]\n\\centering\n\\hspace*{-1.5cm} \n\\begin{tabular}{||c || c | c | c | c | c | c | c | c ||} "
    )
    print("\\hline")
    header = (
        f"{'Label':10} & "
        + " & ".join([f"{language_mapping[lang]:>10}" for lang in all_langauges])
        + f" & {'All languages':>15}"
        + " \\\\ "
    )
    print(header)
    print("\\hline")
    for a in range(4):
        row = df.iloc[a]
        line = (
            f"{(row['sentiment_label'] if a < 3 else 'all'):10} & "
            + " & ".join(
                [f"{row[k]:10.0%}".replace("%", "\%") for k in [*all_langauges, "sum"]]
            )
            + " \\\\"
        )
        if a == 3:
            print("\\hline")
        print(line)
    print("\\hline\\hline")
    print("\end{tabular}")
    print(
        f"\\caption{{Number of tweets split by Sentiments and Languages for Topic {topic}, relative to their Distribution in the overall Dataset.}}"
    )
    print(f"\\label{{table:ov{topic}}}")
    print("\\end{table}")


print_as_table("1")

\begin{table}[H]
\centering
\hspace*{-1.5cm} 
\begin{tabular}{||c || c | c | c | c | c | c | c | c ||} 
\hline
Label      &     German &    English &    Spanish &     French &    Italian &    Russian &  Ukrainian &   All languages \\ 
\hline
negative   &        24\% &        51\% &        35\% &        44\% &        20\% &        52\% &        27\% &        44\% \\
neutral    &       112\% &       147\% &       103\% &       198\% &        95\% &       116\% &        66\% &       133\% \\
positive   &       223\% &       308\% &       149\% &       204\% &       108\% &        98\% &       105\% &       257\% \\
\hline
all        &        69\% &       114\% &        75\% &       102\% &        47\% &        90\% &        62\% &       100\% \\
\hline\hline
\end{tabular}
\caption{Number of tweets split by Sentiments and Languages for Topic 1, relative to their Distribution in the overall Dataset.}
\label{table:ov1}
\end{table}


In [170]:
df_pivot_cur.set_index("sentiment_label").values / (
    df_pivot_all.set_index("sentiment_label").values * (topic_tweets / all_tweets)
)

array([[0.42281426, 1.40871017, 1.65911364, 1.17234223, 0.65083181,
        1.54716347, 0.67830738, 1.23769892],
       [0.75091744, 0.96418344, 1.11764138, 1.0409536 , 0.80169867,
        0.99774207, 0.34171255, 0.90592132],
       [0.12290874, 0.20454932, 0.25876088, 0.40019286, 0.15239726,
        0.1172504 , 0.04114527, 0.20179369],
       [0.5214832 , 1.11374737, 1.28252407, 1.0145451 , 0.64231419,
        1.11037755, 0.38892293, 1.00000011]])

In [245]:
df_pivot_all

,sentiment_label,de,en,es,fr,it,ru,uk,sum
0,negative,903253,6629371,482790,659712,773145,173005,282081,9903357
1,neutral,571670,3993644,407968,211445,295627,239255,443367,6162976
2,positive,105470,1493748,111888,178537,109618,48954,174532,2222747
3,sum,1580393,12116763,1002646,1049694,1178390,461214,899980,18289080


array([0.5214832 , 1.11374719, 1.28252407, 1.0145451 , 0.64231419,
       1.11037755, 0.38892293])

In [118]:
df_agg_all.groupby("lang").sum()["count"].values * (topic_tweets / all_tweets)

array([ 187503.64203146, 1437577.59439539,  118957.61159931,
        124539.55947576,  139808.52657121,   54720.12642165,
        106776.93950522])

In [134]:
# Relative percentagewise change to expected distribution in this topic to all topics
df_agg_all.query("topic==0").groupby("lang").sum()["count"].values / (
    df_agg_all.groupby("lang").sum()["count"].values * (topic_tweets / all_tweets)
)

array([0.5214832 , 1.11374719, 1.28252407, 1.0145451 , 0.64231419,
       1.11037755, 0.38892293])

In [133]:
# Percentagewise distribution of languages in this topic
df_agg_all.query("topic==0").groupby("lang").sum()["count"].values / df_agg_all.query(
    "topic==0"
).groupby("lang").sum()["count"].values.sum()

array([0.04506232, 0.73787262, 0.07031067, 0.05822938, 0.04138516,
       0.0280015 , 0.01913835])

In [132]:
# Percentagewise distribution of languages over all topics
df_agg_all.groupby("lang").sum()["count"].values / df_agg_all.groupby("lang").sum()[
    "count"
].values.sum()

array([0.08641183, 0.66251357, 0.05482211, 0.05739457, 0.06443134,
       0.025218  , 0.04920859])